In [1]:
DESCRIPTION = """Computes IR spectrum using pseudopotentials from Quantum Espresso"""
#Created by Tanmoy & Yuxuan

In [2]:
%load_ext yamlmagic
import numpy as np
import sys
from simtool import DB
from scipy.optimize import curve_fit
import subprocess

In [3]:
import os, stat
from IPython.display import clear_output

try:
    user = str(input())
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user == None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    print("Success")
except:
    print("Something seems wrong with your key")

Success


In [4]:
# These lines import both libraries and then define an array with elements to be used below

#Pymatgen and sub-libraries
import pymatgen
from pymatgen import MPRester, Composition, Element, Structure


#Misc. Tools
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#Import API Key
file = open(os.path.expanduser('~/.mpkey.txt'),"r+")
apikey = file.readline()
file.close()
rester = MPRester(apikey)

In [3]:
%%yaml INPUTS

material:
    type: Text
    value: SnS
    description: Semiconductor to be simulated
        
crystal_spacegroup:
    type: Text
    value: Pnma
    description: Crystal spacegroup   

Energy_cutoff:
    type: Number
    description: Cut off for energy in Ry
    value: 60
    min: 40
    max: 120
    units: rydberg
        
k_points:
    type: Number
    description: Number of k points along a
    value: 2
    min: 1
    max: 10
    
q_points:
    type: Number
    description: Number of q points for DOS calculation
    value: 2
    min: 1
    max: 10

pseudopotential:
    type: List
    value: ['Sn.pbesol-dn-kjpaw_psl.1.0.0.UPF','S.pbesol-n-kjpaw_psl.1.0.0.UPF']
    description: Name of the pseudopotentials

UsageError: Cell magic `%%yaml` not found.


In [2]:
%%yaml OUTPUTS
IR_file:
    type: Text
    description: Name of the IR spectrum output file written by QE
Eigenmodes_file:
    type: Text
    description: Name of the Eigen Modes output file written by QE
Disp_file:
    type: Text
    description: Name of the Dispersion output file written by QE
phDOS_file:
    type: Text
    description: Name of the phonon DOS output file written by QE
IR_list:
    type: Array
    description: IR values with Frequency

UsageError: Cell magic `%%yaml` not found.


In [ ]:
EXTRA_FILES = ["/home/nanohub/paul115/src/irspec/simtool/pseudo"]

In [ ]:
#parameters: define default values which can be overwritten at runtime
#-- parameters should NOT be interdependent
from simtool import getValidatedInputs

defaultInputs = getValidatedInputs(INPUTS)
if defaultInputs:
     globals().update(defaultInputs)

In [ ]:
from numpy import *
data = rester.query({"pretty_formula": material,"spacegroup.symbol": crystal_spacegroup},
                    ["task_id","pretty_formula","nelements","elements","unit_cell_formula","spacegroup.symbol","structure","e_above_hull"])
df= pd.DataFrame.from_dict(data)
sort1 = df.sort_values(["e_above_hull"])
Material_info = sort1.drop_duplicates(["pretty_formula", "spacegroup.symbol"])
Material_info['elements'][0].sort()
pseudopotential[0].sort()

lat1 = data[0]['structure'].as_dict().get('lattice').get('matrix')[0]
lat2 = data[0]['structure'].as_dict().get('lattice').get('matrix')[1]
lat3 = data[0]['structure'].as_dict().get('lattice').get('matrix')[2]

A = data[0]['structure'].as_dict().get('lattice').get('a')
B = data[0]['structure'].as_dict().get('lattice').get('b')
C = data[0]['structure'].as_dict().get('lattice').get('c')

atomconfig_list = []
for i in range(len(data[0]['structure'].as_dict().get('sites'))):
    temp_list = []
    temp_list.append(data[0]['structure'].as_dict().get('sites')[i].get('label'))
    temp_list.extend(data[0]['structure'].as_dict().get('sites')[i].get('abc'))
    atomconfig_list.append(temp_list)
atomconfig = array(atomconfig_list)

numatom=atomconfig.shape[0] #number of atoms in one unit cell
natom=Material_info['nelements'] #Type of atoms
amass=np.zeros(natom[0])
for i in range(natom[0]):
    amass[i]=Element(Material_info['elements'][0][i]).atomic_mass
E_threshold="1.0d-8"

In [ ]:
#Write input files for quantum espresso

import hublib.use


#These values have to be extracted from structure data of materials project


#Write vcrelax input file for qe
f = open("vcrelax.in", "w")
f.write('&CONTROL\n')
f.write('  calculation = "vc-relax"\n')
f.write('  prefix = "{matname}" \n'.format(matname=material))
f.write('  pseudo_dir = "./"\n')
f.write('  outdir = "./"\n')
f.write('  tstress = .true.\n')
f.write('  tprnfor = .true.\n')
f.write('  nstep = 200\n')
f.write('/\n')
f.write('&SYSTEM\n')
f.write('  ibrav = 0, nat = {atom_num}, ntyp = {atom_type}\n'.format(atom_num=numatom,atom_type=natom[0]))
f.write('  occupations = "fixed", \n')
f.write('  ecutwfc = {ecut}\n'.format(ecut=Energy_cutoff))
f.write('/\n')
f.write('&ELECTRONS\n')
f.write('  mixing_mode = "plain"\n')
f.write('  mixing_beta = 0.5\n')
f.write('  startingwfc = "random"\n')
f.write('  conv_thr = {thr}\n'.format(thr=E_threshold))
f.write('/\n')
f.write('&IONS\n')
f.write('/\n')
f.write('&CELL\n')
f.write('  cell_dofree = "all"\n')
f.write('/\n')
f.write('CELL_PARAMETERS angstrom\n')
f.write('  {a1} {a2} {a3}\n'.format(a1=lat1[0],a2=lat1[1],a3=lat1[2]))
f.write('  {b1} {b2} {b3}\n'.format(b1=lat2[0],b2=lat2[1],b3=lat2[2]))
f.write('  {c1} {c2} {c3}\n'.format(c1=lat3[0],c2=lat3[1],c3=lat3[2]))
f.write('ATOMIC_SPECIES\n')
for i in range(natom[0]):
    f.write('  {el} {am} {pp}\n'.format(el=Material_info['elements'][0][i],am=amass[i],pp=pseudopotential[0][i]))
f.write('ATOMIC_POSITIONS (crystal)\n')
f.write('{atcon}\n'.format(atcon=atomconfig))
f.write('K_POINTS (automatic)\n')
f.write('  {kpx} {kpy} {kpz} 0 0 0\n'.format(kpx=k_points,kpy=round(k_points*A/B),kpz=round(k_points*A/C)))
f.close()

#Write ph input file for qe
f = open("phonon.in", "w")
f.write('!Normal modes \n')
f.write('&inputph\n')
f.write('   tr2_ph = 1.0d-14 \n')
f.write('   prefix = "{matname}" \n'.format(matname=material))
for i in range(natom[0]):
    f.write('   amass({ind})='.format(ind=i+1))
    f.write('   {val}\n'.format(val=amass[i]))
f.write('   outdir = "./"\n')
f.write('   epsil = .true.\n')
f.write('   trans = .true.\n')
f.write('   search_sym=.FALSE.\n')
f.write('   asr = .true.\n')
f.write('   fildyn = "dmat.{matname}" \n'.format(matname=material))
f.write('/\n')
f.write('0.0 0.0 0.0\n')
f.close()

#Write dynmat input file for qe
f = open("ircal.in", "w")
f.write('&input fildyn="dmat.{matname}" \n'.format(matname=material))
f.write('  asr="simple" \n')
f.write('/\n')
f.close()

In [ ]:
#Remove unwanted characters from the input files
import re
with open('vcrelax.in', 'r') as f:
    text = f.read()
    pattern = re.sub(r"[\[''\]]", ' ', text)
with open('vcrelax.in', 'w') as my_file:
    my_file.write(pattern)
    f.close()

In [1]:
#Run vcrelax, phonon and dynamt calculation in quantum espresso
dmatfile='dmat.%s' %material
xmlfile='%s.xml' %material
savefile='%s.save' %material
pseudo_list = ""
for i in range(natom[0]):
    pseudo_list += '-i /home/nanohub/paul115/src/irspec/simtool/pseudo/%s ' %pseudopotential[0][i]
pseudo_listp = ""
for i in range(natom[0]):
    pseudo_listp += '/home/nanohub/paul115/src/irspec/simtool/pseudo/%s ' %pseudopotential[0][i]
print(pseudo_list)

import hublib.use
%use espresso-6.2.1
nodes = 4
walltime = '70:00:00'
!submit -n $nodes -w $walltime --runName vcrelax espresso-6.2.1_pw {pseudo_list} -in vcrelax.in

NameError: name 'material' is not defined

In [ ]:
!submit -n $nodes -w $walltime --runName phonon -i {xmlfile} -i {savefile} espresso-6.2.1_ph -in phonon.in {pseudo_listp}

In [ ]:
nodes = 1
walltime = '01:00:00'
!submit -n $nodes -w $walltime --runName ircal -i {dmatfile} espresso-6.2.1_dynmat -in ircal.in

In [ ]:
def search_string_in_file(file_name, string_to_search):
    """Search for the given string in file and return lines containing that string,
    along with line numbers"""
    line_number = 0
    list_of_results = []
    # Open the file in read only mode
    with open(file_name, 'r') as read_obj:
        # Read all lines in the file one by one
        for line in read_obj:
            # For each line, check if line contains the string
            line_number += 1
            if string_to_search in line:
                # If yes, then add the line number & line as a tuple in the list
                list_of_results.append((line_number, line.rstrip()))
    # Return list of tuples containing line numbers and lines where string is found
    return list_of_results

#Delete .xml and .save file from vcrelax
import shutil
os.remove(xmlfile)
shutil.rmtree(savefile)

In [ ]:
#Write scf input file for qe
f = open("scf.in", "w")
f.write('&CONTROL\n')
f.write('  calculation = "scf"\n')
f.write('  prefix = "{matname}" \n'.format(matname=material))
f.write('  pseudo_dir = "./"\n')
f.write('  outdir = "./"\n')
f.write('  tstress = .true.\n')
f.write('  tprnfor = .true.\n')
f.write('/\n')
f.write('&SYSTEM\n')
f.write('  ibrav = 0, nat = {atom_num}, ntyp = {atom_type}\n'.format(atom_num=numatom,atom_type=natom[0]))
#f.write('  ibrav = 0, celldm(1) = {lat_par}, nat = {atom_num}, ntyp = {atom_type}\n'.format(lat_par=lat1[0]*1.88973,atom_num=numatom,atom_type=natom[0]))
f.write('  occupations = "fixed", \n')
f.write('  ecutwfc = {ecut}\n'.format(ecut=Energy_cutoff))
f.write('/\n')
f.write('&ELECTRONS\n')
f.write('  mixing_mode = "plain"\n')
f.write('  mixing_beta = 0.5\n')
f.write('  startingwfc = "random"\n')
f.write('  conv_thr = {thr}\n'.format(thr=E_threshold))
f.write('/\n')
a=search_string_in_file("vcrelax.stdout","Begin final coordinates")
frel = open("vcrelax.stdout", "r")
listItems = frel.read().splitlines()
f.write('CELL_PARAMETERS angstrom\n')
for i in range(3):
    f.write(listItems[i+a[0][0]+4])
    f.write('\n')
f.write('ATOMIC_SPECIES\n')
for i in range(natom[0]):
    f.write('  {el} {am} {pp}\n'.format(el=Material_info['elements'][0][i],am=amass[i],pp=pseudopotential[0][i]))
f.write('ATOMIC_POSITIONS (crystal)\n')
for i in range(numatom):
    f.write(listItems[i+a[0][0]+9])
    f.write('\n')
f.write('K_POINTS (automatic)\n')
f.write('  {kpx} {kpy} {kpz} 0 0 0\n'.format(kpx=k_points,kpy=round(k_points*A/B),kpz=round(k_points*A/C)))
f.close()

#Write ph input file for DOS & dispersion
f = open("phonon2.in", "w")
f.write('!Normal modes \n')
f.write('&inputph\n')
f.write('   tr2_ph = 1.0d-14 \n')
f.write('   prefix = "{matname}" \n'.format(matname=material))
for i in range(natom[0]):
    f.write('   amass({ind})='.format(ind=i+1))
    f.write('   {val}\n'.format(val=amass[i]))
f.write('   outdir = "./"\n')
f.write('   ldisp = .true.\n')
f.write('   search_sym=.FALSE.\n')
f.write('   nq1 = {qp}, nq2 = {qp}, nq3= {qp}\n'.format(qp=q_points))
f.write('   fildyn = "{matname}.dyn" \n'.format(matname=material))
f.write('/\n')
f.close()

#Write q2r input file for dos and dispersion
f = open("q2r.in", "w")
f.write('&input fildyn="{matname}.dyn", zasr="simple", flfrc="{matname}.fc" \n'.format(matname=material))
f.write('/\n')
f.close()

#Write matdyn input file for dispersion calculation
f = open("matdyn.in", "w")
f.write('&input \n')
f.write(' asr = "simple",  \n')
for i in range(natom[0]):
    f.write(' amass({ind})='.format(ind=i+1))
    f.write(' {val}\n'.format(val=amass[i]))
f.write(' flfrc="{matname}.fc", flfrq="{matname}.freq", q_in_band_form=.true. \n'.format(matname=material))
f.write(' q_in_cryst_coord = .true.  \n')
f.write('/\n')
f.write('5 \n')
f.write(' 0.0000 0.0000 0.0000 30 !Gamma \n')
f.write(' -0.500 0.0000 -0.500 10 !X \n')
f.write(' 0.0000 0.3750 -0.375 20 \n')
f.write(' 0.0000 0.0000 0.0000 30 \n')
f.write(' 0.0000 0.5000 0.0000 1 \n')
f.write('/\n')
f.close()

#Write matdyn input file for phdos calculation 
f = open("phdos.in", "w")
f.write('&input \n')
f.write(' asr = "simple", dos=.true. \n')
for i in range(natom[0]):
    f.write(' amass({ind})='.format(ind=i+1))
    f.write(' {val}\n'.format(val=amass[i]))
f.write(' flfrc="{matname}.fc", fldos="{matname}.phdos", nk1=40,nk2=40,nk3=40 \n'.format(matname=material))
f.write('/\n')
f.close()

#remove unwanted character from scf file
import re
with open('scf.in', 'r') as f:
    text = f.read()
    pattern = re.sub(r"[\[''\]]", ' ', text)
with open('scf.in', 'w') as my_file:
    my_file.write(pattern)
    f.close()

In [ ]:
import hublib.use
%use espresso-6.2.1
nodes = 4
walltime = '70:00:00'
!submit -n $nodes -w $walltime --runName scf espresso-6.2.1_pw {pseudo_list} -in scf.in

In [ ]:
!submit -n $nodes -w $walltime --runName phonon2 -i {xmlfile} -i {savefile} espresso-6.2.1_ph -in phonon2.in {pseudo_listp}

In [ ]:
a=search_string_in_file("phonon2.stdout","Calculation of q")
qsize=array(a).shape[0]
dyn_list = ""
for i in range(qsize+1):
    dyn_list += '-i %s.dyn' %material
    dyn_list += '%s ' %i

walltime = '06:00:00'
nodes = 1
!submit -n $nodes -w $walltime --runName  q2r {dyn_list} espresso-6.2.1_q2r -in q2r.in

In [ ]:
fc_file='%s.fc'%material
!submit -n $nodes -w $walltime --runName  matdyn -i {fc_file} espresso-6.2.1_matdyn -in matdyn.in

In [ ]:
!submit -n $nodes -w $walltime --runName  phdos -i {fc_file} espresso-6.2.1_matdyn -in phdos.in

In [ ]:
line=search_string_in_file("ircal.stdout","# mode")
f = open("ircal.stdout", "r")
listItems = f.read().splitlines()
IRlist=np.zeros([3*numatom, 4])
for i in range(3*numatom):
   IRlist[i]=listItems[i+line[0][0]].split()
f.close()

In [ ]:
db = DB(OUTPUTS)
with open("ircal.stdout") as fp:
    db.save('IR_file', fp.read())
with open("dynmat.axsf") as fp:
    db.save('Eigenmodes_file', fp.read())
with open("%s.freq" %material) as fp:
    db.save('Disp_file', fp.read())
with open("%s.phdos" %material) as fp:
    db.save('phDOS_file', fp.read())
    
db.save('IR_list', IRlist)